In [40]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Creating data generator to process the input

In [31]:


# Define your dataset directory
dataset_dir = 'dataset'

# Define image size and batch size
img_height, img_width = 640, 640
batch_size = 32

# Create an ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting data into training and validation sets
)

# Load and prepare training data
train_data = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Load and prepare validation data
val_data = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)



Found 1050 images belonging to 10 classes.
Found 258 images belonging to 10 classes.


# Model 1 - Trying Custom CNN based architecture

In [32]:
# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(10, activation='softmax')  # num_classes should be the number of classes in your dataset
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])




In [33]:
# Train the model
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)

Epoch 1/10


33/33 [==============================] - 269s 7s/step - loss: 3.7763 - accuracy: 0.1276 - val_loss: 1.9668 - val_accuracy: 0.2054
Epoch 2/10
33/33 [==============================] - 207s 6s/step - loss: 1.6604 - accuracy: 0.3381 - val_loss: 1.4734 - val_accuracy: 0.3256
Epoch 3/10
33/33 [==============================] - 201s 6s/step - loss: 1.4567 - accuracy: 0.4343 - val_loss: 1.1210 - val_accuracy: 0.5581
Epoch 4/10
33/33 [==============================] - 198s 6s/step - loss: 0.9841 - accuracy: 0.5943 - val_loss: 0.7517 - val_accuracy: 0.7054
Epoch 5/10
33/33 [==============================] - 200s 6s/step - loss: 0.8413 - accuracy: 0.6743 - val_loss: 0.9315 - val_accuracy: 0.5581
Epoch 6/10
33/33 [==============================] - 198s 6s/step - loss: 0.7691 - accuracy: 0.6943 - val_loss: 0.7089 - val_accuracy: 0.7403
Epoch 7/10
33/33 [==============================] - 204s 6s/step - loss: 0.5943 - accuracy: 0.7638 - val_loss: 0.7009 - val_accuracy: 0.7016
Epoch 8/10


In [34]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 638, 638, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 319, 319, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 317, 317, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 158, 158, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 156, 156, 128)     73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 78, 78, 128)      

In [35]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

In [36]:

# Define your dataset directory
dataset_dir = 'dataset'

# Define image size
img_height, img_width = 640, 640
num_classes = 10  # Specify the number of classes in your dataset

# Model 2 - Trying pretrained VGG16 with imagenet weights

In [37]:


# Load the pre-trained VGG16 model (without including the top FC layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Train the model (similar to previous code)


In [38]:
# Display the model summary
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 640, 640, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 640, 640, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 640, 640, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 320, 320, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 320, 320, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 320, 320, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 160, 160, 128)     0     

In [39]:
# Train the model
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)

Epoch 1/10
33/33 [==============================] - 551s 16s/step - loss: 14.0596 - accuracy: 0.3638 - val_loss: 1.0653 - val_accuracy: 0.6589
Epoch 2/10
33/33 [==============================] - 532s 16s/step - loss: 1.2458 - accuracy: 0.5314 - val_loss: 0.8239 - val_accuracy: 0.7248
Epoch 3/10
33/33 [==============================] - 540s 16s/step - loss: 1.0324 - accuracy: 0.6048 - val_loss: 1.1012 - val_accuracy: 0.7093
Epoch 4/10
 6/33 [====>.........................] - ETA: 5:38 - loss: 0.8222 - accuracy: 0.6882

KeyboardInterrupt: 